 # 📌 PipeLine para descarga y atomización de Datos

🔹Este documento describe la arquitectura utilizada para descargar datos crudos desde KoboToolbox, almacenarlos en Google Cloud Storage, y posteriormente atomizarlos y cargarlos en BigQuery mediante Cloud Functions.


-------------------
### 📁 Perfiladores de Corriente 
- La estructura de Cloud Functions requiere de dos archivos: **main.py** y **requirement.txt**
- *main.py* - Código con la función para acceder a los datos de la aplicación KoboToolbox y guardar los registros (datos crudos) en un Bucket de *Cloud Storage*
- *requirement.txt* - Debe de contener:

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= Perfiladores_de_Corriente

#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/akth74ZsstPt82WXhLUKmN/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:
            # Extraer datos del grupo "group_dr3zv09" (Información general)
            grupo_principal = item.get("group_dr3zv09", {})
        
            # Extraer datos del grupo "group_jf5li71" (Información del sensor)
            grupo_sensor = item.get("group_jf5li71", {})
            data_limpia.append({
                
                # --- Información general (grupo_dr3zv09) ---
                "nombre_completo": grupo_principal.get("Nombre_Completo"),
                "fecha_recoleccion": grupo_principal.get("Fecha_y_hora_de_recolecci_n"),
                "reserva": grupo_principal.get("reserva"),
                "sitio": grupo_principal.get("sitio"),
                "codigo_sitio": grupo_principal.get("codigo"),
                
                # --- Información del sensor (grupo_jf5li71) ---
                "coordenadas_sensor": coordenadas,
                "frecuencia_operacion": grupo_sensor.get("Frecuencia_de_operaci_n"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "Perfiladores_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource Perfiladores_de_Corriente \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````


## 📁 Parámetros de Agua
--------

### ⚡ Descarga de datos crudos
- *requirement.txt* 

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= Parámetros_de_Agua
#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/aZcdmftcBtnpfW8HvV78as/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------


### 📋 Atomización de datos
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:
            # Extraer datos del grupo "group_dr3zv09" (Información general)
            grupo_principal = item.get("group_dr3zv09", {})
        
            # Extraer datos del grupo "group_jf5li71" (Información de calidad de agua)
            grupo_agua = item.get("group_kv0zx70", {})
            data_limpia.append({
                
                # --- Información general (grupo_dr3zv09) ---
                "nombre_completo": grupo_principal.get("Nombre_Completo"),
                "fecha_recoleccion": grupo_principal.get("Fecha_y_hora_de_recolecci_n"),
                "reserva": grupo_principal.get("reserva"),
                "sitio": grupo_principal.get("sitio"),
                "codigo_sitio": grupo_principal.get("codigo"),
                
                # --- Información de calidad de agua (grupo_kv0zx70) ---
                "Temperatura": grupo_agua.get("Temp_C"),  
                "Salinidad": grupo_agua.get("Salinidad_PSU"),
                "Conductividad": grupo_sensor.get("Conductividad_mS_cm"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "Parametros_agua_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource Parámetros_de_Agua \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````


## 📁 Cámaras Terrestre
--------

### ⚡ Descarga de datos crudos
- *requirement.txt* 

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= Cámaras_Terrestres
#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/aMQsvne8A7orHTdj6Yfh5x/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------


### 📋 Atomización de datos
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:

            # Extraer datos del grupo "group_kv0zx70" (Información de instalación)
             grupo_intalación = item.get("group_kv0zx70", {})

            # Extraer datos del grupo "group_dr3zv09" (Información general)
            grupo_principal = item.get("group_dr3zv09", {})
        
            # Extraer datos del grupo "group_rw9dd47/group_it8nx96" (Información de OP)
            grupo_op = item.get("group_kv0zx70", {})
            data_limpia.append({
                
                # --- Información general (grupo_dr3zv09) ---
                "nombre_completo": grupo_principal.get("Nombre_Completo"),
                "fecha_recoleccion": grupo_principal.get("Fecha_y_hora_de_recolecci_n"),
                "reserva": grupo_principal.get("reserva"),
                "sitio": grupo_principal.get("sitio"),
                "codigo_sitio": grupo_principal.get("codigo"),
                
                # --- Información de operación y mantenimiento (group_rw9dd47/group_it8nx96) ---
                "Modo de Operación": grupo_op.get("Modo_de_operaci_n_001"),  
                "Señal": grupo_op.get("Intensidad_de_la_se_al"),

                # ---- Información de Instalación (group_kv0zx70)

                "Altura del sensor": grupo_instlación.get("Altura_del_sensor_m_desde_el"),  
                "Coordenadas": grupo_op.get("Latitud_y_Longitud_del_sensor"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "Cámaras_terrestre_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource Cámaras_Terrestres \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````


## 📁 Zooplancton
--------

### ⚡ Descarga de datos crudos
- *requirement.txt* 

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= Zooplancton
#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/aojT2JZfgdqjTsxNszRdVz/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------


### 📋 Atomización de datos
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:
            # Extraer datos del grupo "group_kv0zx70" (Información de la muestra)
            grupo_muestra = item.get("group_kv0zx70", {})
        
            # Extraer datos del grupo "group_jf5li71" (Información de registro CTD)
            grupo_registro = item.get("group_zq1vg22", {})
            data_limpia.append({
                
                # --- Información de la muestras (group_kv0zx70) ---
                "ID Muestra": grupo_muestra.get("ID_Muestra"),
                "Estado de la Marea": grupo_muestra.get("Estado_de_la_marea"),
            
                
                # --- Información de registro de CTD (group_zq1vg22) ---
                "Temperatura": grupo_registro.get("Temp_C"),  
                "Salinidad": grupo_registro.get("Salinidad_ppt"),
                "Profundidad": grupo_registro.get("Profundidad_m"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "Zooplancton_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource Zooplancton \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````


## 📁 Vuelo de dron
--------

### ⚡ Descarga de datos crudos
- *requirement.txt* 

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= Vuelo_de_dron
#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/aBwshXwrJ77EyRLtrx2s8z/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------


### 📋 Atomización de datos
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:
            # Extraer datos del grupo "group_dr3zv09" (Información general)
            grupo_principal = item.get("group_dr3zv09", {})
        
            # Extraer datos del grupo "group_kv0zx70" (Información del vuelo de dron)
            grupo_vuelo = item.get("group_kv0zx70", {})
            data_limpia.append({
                
                # --- Información general (grupo_dr3zv09) ---
                "nombre_completo": grupo_principal.get("Nombre_Completo"),
                "fecha_recoleccion": grupo_principal.get("Fecha_y_hora_de_recolecci_n"),
                "reserva": grupo_principal.get("reserva"),
                "sitio": grupo_principal.get("sitio"),
                "codigo_sitio": grupo_principal.get("codigo"),
                
                # --- Información de calidad de agua (grupo_kv0zx70) ---
                "Tipo de sensor": item.get("Tipo_de_sensor"),  # Fuera de grupo
                "Indice": grupo_vuelo.get("_ndice"),
                "Conductividad": grupo_vuelo.get("Conductividad_mS_cm"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "Vuelo_de_dron_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource Vuelo_de_dron \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````


## 📁 Estaciones Meteorológicas
--------

### ⚡ Descarga de datos crudos
- *requirement.txt* 

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= Estaciones_Meteorológicas 
#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/an57S6QY6v3RwTWDnM2ev4/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------


### 📋 Atomización de datos
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:
            # Extraer datos del grupo "group_dr3zv09" (Información general)
            grupo_principal = item.get("group_dr3zv09", {})
        
            # Extraer datos del grupo "group_jf5li71" (Información de calidad de agua)
            grupo_instalacion = item.get("group_kv0zx70", {})
            data_limpia.append({
                
                # --- Información general (grupo_dr3zv09) ---
                "nombre_completo": grupo_principal.get("Nombre_Completo"),
                "fecha_recoleccion": grupo_principal.get("Fecha_y_hora_de_recolecci_n"),
                "reserva": grupo_principal.get("reserva"),
                "sitio": grupo_principal.get("sitio"),
                "codigo_sitio": grupo_principal.get("codigo"),
                
                # --- Información de instalación de sensor (grupo_kv0zx70) ---
                "Tipo de sensor": item.get("Tipo_de_sensor"),  # Fuera de grupo
                "Coordenadas": grupo_instalacion.get("Latitud_y_Longitud_del_sensor"),
                "Altura del sensor": grupo_instalacion.get("Altura_del_sensor_m_desde_el"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "Estaciones_meteorológicas_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource Estaciones_Meteorológicas \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````


## 📁 CTD's
--------

### ⚡ Descarga de datos crudos
- *requirement.txt* 

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= CTDS
#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/aAjboEpfSEbFjTZBxF8CKR/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------


### 📋 Atomización de datos
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:
            # Extraer datos del grupo "group_dr3zv09" (Información general)
            grupo_principal = item.get("group_dr3zv09", {})
        
            # Extraer datos del grupo "group_wh77o68" (Información de OP)
            grupo_op = item.get("group_wh77o68", {})
            data_limpia.append({
                
                # --- Información general (grupo_dr3zv09) ---
                "nombre_completo": grupo_principal.get("Nombre_Completo"),
                "fecha_recoleccion": grupo_principal.get("Fecha_y_hora_de_recolecci_n"),
                "reserva": grupo_principal.get("reserva"),
                "sitio": grupo_principal.get("sitio"),
                "codigo_sitio": grupo_principal.get("codigo"),
                
                # --- Información de OP (group_wh77o68) ---
                "Tipo de sensor": item.get("Tipo_de_sensor"),  # Fuera de grupo
                "Mantenimiento a": grupo_op.get("Mantenimiento_a"),
                "¿Necesita ser recuperado?": grupo_op.get("_El_sensor_necesita_ser_recupe"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "CTDS_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource CTDS \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````


## 📁 Estructura Forestal 
--------

### ⚡ Descarga de datos crudos
- *requirement.txt* 

 

1) requests → Para hacer solicitudes HTTP a la API de KoBoToolbox.

2) google.cloud.storage → Para conectarse al Bucket de Cloud Storage.

3) functions_framework → Permite ejecutar la función como Cloud Function.

Para configurar la función en Google Cloud Functions (GCF), es necesario establecer las variables de entorno requeridas por el código principal, lo que simplifica su despliegue y mantenimiento.

- KOBO_TOKEN= "d0e1084994983390bd6f50e3ee61e9c522aee152"
- BUCKET_NAME= Estructura_forestal
#### 🚀 Código *main.py*
```` python
import os
import json
import requests
from google.cloud import storage
from datetime import datetime
import functions_framework

@functions_framework.http # Permite que la función se ejecute cuando recibe una petición web y convierte la función en una Cloud Function HTTP
def main(request):
    
    # --- Configuración ---
    KOBO_URL = "https://kf.kobotoolbox.org/api/v2/assets/aenA9VYsH68Csi8QkDCntr/data.json" #Endpoint específico de la API de KoboToolbox para un asset/proyecto
    KOBO_TOKEN = os.getenv("KOBO_TOKEN")
    BUCKET_NAME = os.getenv("BUCKET_NAME")

    # --- Descargar datos desde KoboToolbox ---
    headers = {"Authorization": f"Token {KOBO_TOKEN}"} # Configura autenticación Bearer Token para la API
    response = requests.get(KOBO_URL, headers=headers) # Ejecuta la petición GET y almacena la respuesta

    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}", response.status_code

    data = response.json() # Convertir respuesta a JSON

    # --- Nombrar archivo por fecha ---
    fecha = datetime.utcnow().strftime("%Y-%m-%dT%H%M%SZ") # Se obtiene la fecha y se le da formato
    filename = f"kobo_raw_{fecha}.json" # Se nombra el archivo

    # --- Guardar archivo en Cloud Storage ---
    
    storage_client = storage.Client() # Crea una instancia del cliente oficial de Python para interactuar con Google Cloud Storage 
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(filename) # Representa el archivo que se creará en el bucket
    blob.upload_from_string(json.dumps(data), content_type="application/json")

    print(f"Archivo subido: gs://{BUCKET_NAME}/{filename}")
    return f"Datos guardados en {filename}", 200

````

🔹Para realizar la automatización de la descarga de datos se tiene que declarar un mensaje de activación, el disparador (trigger) y por ultimo se declaran las condiciones para que se ejecute en automatico. 

1. Mensaje de activación funciona como intermediario entre Scheduler y Cloud Function:\
   gcloud pubsub topics create kobo-extractor-trigger
   
2. Trrigger despliega una función que se ejecuta automáticamente cuando llega un mensaje al topic: \
   gcloud functions deploy extraer_datos_kobo \
  --runtime python311 \
  --trigger-topic kobo-extractor-trigger \
  --region=us-central1

   
4. Programar la tarea: \
   gcloud scheduler jobs create pubsub ejecutar-extraccion-kobo \
  --schedule="0 8 1 * *" \
  --topic=kobo-extractor-trigger \
  --message-body="Iniciar extracción de KoboToolbox" \
  --time-zone="America/Mexico_City"


----------------------


### 📋 Atomización de datos
Una vez generada la dase de datos cruda en Cloud Storage se despliega otra función donde a partir de los datos crudos se haga la atomización de los registros y se genere el Query. Su respectivo requirements.txt debe de contener: 
1) google-cloud-storage
2) google-cloud-bigquery


#### 🚀 Código *main.py*
```` python
    import json
    from google.cloud import storage, bigquery

    def atomizar_y_cargar(event, context):
        """Se ejecuta automáticamente cuando se sube un nuevo archivo al bucket."""
        bucket_name = event['bucket']
        file_name = event['name']
        print(f"Nuevo archivo detectado: gs://{bucket_name}/{file_name}")

        # --- Leer el archivo desde el bucket ---
        
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name) 
        blob = bucket.blob(file_name) # Referencia al archivo específico
        raw_data = blob.download_as_text()
        json_data = json.loads(raw_data) #Convierte el texto JSON a objeto Python

        # --- Extraer los registros del JSON ---
        registros = json_data.get("results", [])

        # --- Limpiar y estructurar los datos ---
        data_limpia = []
        
        for item in registros:
            # Extraer datos del grupo "group_dr3zv09" (Información general)
            grupo_principal = item.get("group_dr3zv09", {})
        
            # Extraer datos del grupo "group_rh5ho64/group_yh9gj88" (Medidas de Arbol)
            grupo_arbol = item.get("group_rh5ho64/group_yh9gj88", {})
            data_limpia.append({
                
                # --- Información general (grupo_dr3zv09) ---
                "nombre_completo": grupo_principal.get("Nombre_Completo"),
                "fecha_recoleccion": grupo_principal.get("Fecha_y_hora_de_recolecci_n"),
                "reserva": grupo_principal.get("reserva"),
                "sitio": grupo_principal.get("sitio"),
                "codigo_sitio": grupo_principal.get("codigo"),
                
                # --- Información de medidas de Arbol (group_wh77o68) ---
                "Tipo de encuesta": item.get("Tipo_de_encuesta"),  # Fuera de grupo
                "Especies Individuales": grupo_arbol.get("Especies_individuales"),
                "Altura del árbol": grupo_op.get("Altura_del_individuo_m"),

        })


        # --- Subir los datos a BigQuery ---
        bq_client = bigquery.Client()
        table_id = "Estructura_forestal_atomizado"

        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # agrega datos nuevos sin borrar los anteriores
        autodetect=True # BigQuery infiere automáticamente el esque de datos
        )

        job = bq_client.load_table_from_json(data_limpia, table_id, job_config=job_config)
        job.result()  # Esperar a que finalice la carga, dtiene la ejecución en CloudFunctions

        print(f"{len(data_limpia)} registros cargados a {table_id}")

````

Desde Terminal se generá el ***Disparador*** de la función para que se relice de forma automatica cada vez que se detecten nuevos datos en el bucket

````pyhton 
gcloud functions deploy atomizar_y_cargar \
  --runtime python311 
  --trigger-resource Estructura_forestal \
  --trigger-event google.storage.object.finalize \
  --region=us-central1
````
